In [1]:
using Plots
using PyCall
using ControlSystems
#using PyPlot

gr()

Plots.GRBackend()

In [2]:
zc=0.22
g=9.8
dt=0.0025
L=0.15
x=0.05


#support time
Tsup=Int32(0.7/dt)
#both foot support point time
Tdl=Int32(0.1/dt)

#total step
k_max=Int32(1+(Tsup*3)+(Tdl*2))
#preview step
M=400

#support point
#sx=[0.0 0.0 0.3 0.6 0.9 0.9]
#sy=[0.0 0.2 0.0 0.2 0.0 0.2]

#support point
real_sx=[0.0; (1/2)*sqrt(3)*L; (1/2)*sqrt(3)*L+x; (1/2)*sqrt(3)*L+x; sqrt(3)*L+x; sqrt(3)*L+2x]
real_sy=[L/2; 0.0; L; L/2; 0.0; L; L/2; 0.0]

#Virtual support point
sx=[(1/2)*(sqrt(3)*L+x) 
    (1/2)*sqrt(3)*L+x 
    (3/4)*sqrt(3)*L+x ]
sy=[L/2 
    3*L/4 
    L/4 ]


#to generate trajectory of ZMP when both foot support point 
x_tdl_a=zeros(Float64,2)
x_tdl_b=zeros(Float64,2)
y_tdl_a=zeros(Float64,2)
y_tdl_b=zeros(Float64,2)

for i in 1:2
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:2
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#target ZMP vector
px_ref=[[sx[1] for s in 1:Tsup]
        [x_tdl_a[1]*s+x_tdl_b[1] for s in 1*Tsup*dt+(1-1)*Tdl*dt:dt:1*Tsup*dt+(1-1)*Tdl*dt+Tdl*dt-dt]
        [sx[2] for s in 1:Tsup]
        [x_tdl_a[2]*s+x_tdl_b[2] for s in 2*Tsup*dt+(2-1)*Tdl*dt:dt:2*Tsup*dt+(2-1)*Tdl*dt+Tdl*dt-dt]
        [sx[3] for s in 1:Tsup+1+M]]
        

py_ref=[[sy[1] for s in 1:Tsup]
    [y_tdl_a[1]*s+y_tdl_b[1] for s in 1*Tsup*dt+(1-1)*Tdl*dt:dt:1*Tsup*dt+(1-1)*Tdl*dt+Tdl*dt-dt]
        [sy[2] for s in 1:Tsup]
    [y_tdl_a[2]*s+y_tdl_b[2] for s in 2*Tsup*dt+(2-1)*Tdl*dt:dt:2*Tsup*dt+(2-1)*Tdl*dt+Tdl*dt-dt]
        [sy[3] for s in 1:Tsup+1+M]]
   

#computed ZMP vector
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max);

In [3]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[30000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#state vector
x=[ sx[1]+0.001
    0.0
    0.0]

y=[ L/2
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0];

In [4]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #to record for plot
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #to record for plot
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 


xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y


dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR


old_x=x
old_y=y

xu=0
yu=0

P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 0.0

In [5]:
for i in 1:k_max-1  
    
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]
    
    k+=1
    
    x_plot_log[k+1]=x[1] #to record for plot
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #to record for plot
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    
    dx=x-old_x
    dy=y-old_y
    
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    
    
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y
    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    
    old_x=x
    old_y=y
    
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end
    

In [6]:
plot(t,px_ref,lab="Target ZMP",linewidth=3,linecolor="black")
plot!(t,x_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t,px,lab="Target ZMP",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP_x[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 time[ms] 
 
 
 ZMP 
 
 
 x[m] 
 
<polyline clip-path="url(#clip2902)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,142.727 39.9709,142.727 40.5717,142.727 41.1725,142.727 41.7734,142.727 42.3742,142.727 42.975,142.727 43.5758,142.727 44.1767,142.727 44.7775,142.727 
 45.3783,142.727 45.9791,142.727 46.5799,142.727 47.1808,142.727 47.7816,142.727 48.3824,142.727 48.9832,142.727 49.584,142.727 50.1849,142.727 50.7857,142.727 
 51.3865,142.727 51.9873,142.727 52.5882,142.727 53.189,142.727 53.7898,142.727 54.3906,142.727 54.9914,142.727 55.5923,142.727 56.1931,142.727 56.7939,142.727 
 57.3947,142.727 57.9955,142.727 58.5964,142.727 59.1972,142.727 59.798,142.727 60.3988,142.727 60.9997,142.727 61.6005,142.727 62.2013,142.727 62.8021,142.727 
 63.4029,142.727 64.0038,142.727 64.6046,142.727 65.2054,142.727 65.8062,142.727 66.4071,142.727 67.0079,142.727 67.6087,142.727 68.2095,142.727 68.8103,142.727 
 69.4112,142.727 70.012,142.727 70.6128,142.727 71.2136,142.727 71.8144,142.727 72.4153,142.727 73.0161,142.727 73.6169,142.727 74.2177,142.727 74.8186,142.727 
 75.4194,142.727 76.0202,142.727 76.621,142.727 77.2218,142.727 77.8227,142.727 78.4235,142.727 79.0243,142.727 79.6251,142.727 80.226,142.727 80.8268,142.727 
 81.4276,142.727 82.0284,142.727 82.6292,142.727 83.2301,142.727 83.8309,142.727 84.4317,142.727 85.0325,142.727 85.6333,142.727 86.2342,142.727 86.835,142.727 
 87.4358,142.727 88.0366,142.727 88.6375,142.727 89.2383,142.727 89.8391,142.727 90.4399,142.727 91.0407,142.727 91.6416,142.727 92.2424,142.727 92.8432,142.727 
 93.444,142.727 94.0448,142.727 94.6457,142.727 95.2465,142.727 95.8473,142.727 96.4481,142.727 97.049,142.727 97.6498,142.727 98.2506,142.727 98.8514,142.727 
 99.4522,142.727 100.053,142.727 100.654,142.727 101.255,142.727 101.856,142.727 102.456,142.727 103.057,142.727 103.658,142.727 104.259,142.727 104.86,142.727 
 105.46,142.727 106.061,142.727 106.662,142.727 107.263,142.727 107.864,142.727 108.465,142.727 109.065,142.727 109.666,142.727 110.267,142.727 110.868,142.727 
 111.469,142.727 112.069,142.727 112.67,142.727 113.271,142.727 113.872,142.727 114.473,142.727 115.074,142.727 115.674,142.727 116.275,142.727 116.876,142.727 
 117.477,142.727 118.078,142.727 118.679,142.727 119.279,142.727 119.88,142.727 120.481,142.727 121.082,142.727 121.683,142.727 122.283,142.727 122.884,142.727 
 123.485,142.727 124.086,142.727 124.687,142.727 125.288,142.727 125.888,142.727 126.489,142.727 127.09,142.727 127.691,142.727 128.292,142.727 128.893,142.727 
 129.493,142.727 130.094,142.727 130.695,142.727 131.296,142.727 131.897,142.727 132.497,142.727 133.098,142.727 133.699,142.727 134.3,142.727 134.901,142.727 
 135.502,142.727 136.102,142.727 136.703,142.727 137.304,142.727 137.905,142.727 138.506,142.727 139.106,142.727 139.707,142.727 140.308,142.727 140.909,142.727 
 141.51,142.727 142.111,142.727 142.711,142.727 143.312,142.727 143.913,142.727 144.514,142.727 145.115,142.727 145.716,142.727 146.316,142.727 146.917,142.727 
 147.518,142.727 148.119,142.727 148.72,142.727 149.32,142.727 149.921,142.727 150.522,142.727 151.123,142.727 151.724,142.727 152.325,142.727 152.925,142.727 
 153.526,142.727 154.127,142.727 154.728,142.727 155.329,142.727 155.929,142.727 156.53,142.727 157.131,142.727 157.732,142.727 158.333,142.727 158.934,142.727 
 159.534,142.727 160.135,142.727 160.736,142.727 161.337,142.727 161.938,142.727 162.539,142.727 163.139,142.727 163.74,142.727 164.341,142.727 164.942,142.727 
 165.543,142.727 166.143,142.727 166.744,142.727 167.345,142.727 167.946,142.727 168.547,142.727 169.148,142.727 169.748,142.727 170.349,142.727 170.95,142.727 
 171.551,142.727 172.152,142.727 172.752,142.727 173.353,142.727 173.954

In [7]:
plot(t,py_ref,lab="Target ZMP y",linewidth=3,linecolor="black")
plot!(t,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t,py,lab="ZMP y",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 time[ms] 
 
 
 ZMP[m] 
 
<polyline clip-path="url(#clip3002)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,131.258 39.9709,131.258 40.5717,131.258 41.1725,131.258 41.7734,131.258 42.3742,131.258 42.975,131.258 43.5758,131.258 44.1767,131.258 44.7775,131.258 
 45.3783,131.258 45.9791,131.258 46.5799,131.258 47.1808,131.258 47.7816,131.258 48.3824,131.258 48.9832,131.258 49.584,131.258 50.1849,131.258 50.7857,131.258 
 51.3865,131.258 51.9873,131.258 52.5882,131.258 53.189,131.258 53.7898,131.258 54.3906,131.258 54.9914,131.258 55.5923,131.258 56.1931,131.258 56.7939,131.258 
 57.3947,131.258 57.9955,131.258 58.5964,131.258 59.1972,131.258 59.798,131.258 60.3988,131.258 60.9997,131.258 61.6005,131.258 62.2013,131.258 62.8021,131.258 
 63.4029,131.258 64.0038,131.258 64.6046,131.258 65.2054,131.258 65.8062,131.258 66.4071,131.258 67.0079,131.258 67.6087,131.258 68.2095,131.258 68.8103,131.258 
 69.4112,131.258 70.012,131.258 70.6128,131.258 71.2136,131.258 71.8144,131.258 72.4153,131.258 73.0161,131.258 73.6169,131.258 74.2177,131.258 74.8186,131.258 
 75.4194,131.258 76.0202,131.258 76.621,131.258 77.2218,131.258 77.8227,131.258 78.4235,131.258 79.0243,131.258 79.6251,131.258 80.226,131.258 80.8268,131.258 
 81.4276,131.258 82.0284,131.258 82.6292,131.258 83.2301,131.258 83.8309,131.258 84.4317,131.258 85.0325,131.258 85.6333,131.258 86.2342,131.258 86.835,131.258 
 87.4358,131.258 88.0366,131.258 88.6375,131.258 89.2383,131.258 89.8391,131.258 90.4399,131.258 91.0407,131.258 91.6416,131.258 92.2424,131.258 92.8432,131.258 
 93.444,131.258 94.0448,131.258 94.6457,131.258 95.2465,131.258 95.8473,131.258 96.4481,131.258 97.049,131.258 97.6498,131.258 98.2506,131.258 98.8514,131.258 
 99.4522,131.258 100.053,131.258 100.654,131.258 101.255,131.258 101.856,131.258 102.456,131.258 103.057,131.258 103.658,131.258 104.259,131.258 104.86,131.258 
 105.46,131.258 106.061,131.258 106.662,131.258 107.263,131.258 107.864,131.258 108.465,131.258 109.065,131.258 109.666,131.258 110.267,131.258 110.868,131.258 
 111.469,131.258 112.069,131.258 112.67,131.258 113.271,131.258 113.872,131.258 114.473,131.258 115.074,131.258 115.674,131.258 116.275,131.258 116.876,131.258 
 117.477,131.258 118.078,131.258 118.679,131.258 119.279,131.258 119.88,131.258 120.481,131.258 121.082,131.258 121.683,131.258 122.283,131.258 122.884,131.258 
 123.485,131.258 124.086,131.258 124.687,131.258 125.288,131.258 125.888,131.258 126.489,131.258 127.09,131.258 127.691,131.258 128.292,131.258 128.893,131.258 
 129.493,131.258 130.094,131.258 130.695,131.258 131.296,131.258 131.897,131.258 132.497,131.258 133.098,131.258 133.699,131.258 134.3,131.258 134.901,131.258 
 135.502,131.258 136.102,131.258 136.703,131.258 137.304,131.258 137.905,131.258 138.506,131.258 139.106,131.258 139.707,131.258 140.308,131.258 140.909,131.258 
 141.51,131.258 142.111,131.258 142.711,131.258 143.312,131.258 143.913,131.258 144.514,131.258 145.115,131.258 145.716,131.258 146.316,131.258 146.917,131.258 
 147.518,131.258 148.119,131.258 148.72,131.258 149.32,131.258 149.921,131.258 150.522,131.258 151.123,131.258 151.724,131.258 152.325,131.258 152.925,131.258 
 153.526,131.258 154.127,131.258 154.728,131.258 155.329,131.258 155.929,131.258 156.53,131.258 157.131,131.258 157.732,131.258 158.333,131.258 158.934,131.258 
 159.534,131.258 160.135,131.258 160.736,131.258 161.337,131.258 161.938,131.258 162.539,131.258 163.139,131.258 163.74,131.258 164.341,131.258 164.942,131.258 
 165.543,131.258 166.143,131.258 166.744,131.258 167.345,131.258 167.946,131.258 168.547,131.258 169.148,131.258 169.748,131.258 170.349,131.258 170.95,131.258 
 171.551,131.258 172.152,131.258 172.752,131.258 173.353,131.258 173.954,13

In [8]:
plot(t,xv_plot_log,lab="Velocity x",linewidth=3)
plot!(xlab="time[ms]",ylab="Velocity[m/s]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 0.00 
 
 
 0.03 
 
 
 0.06 
 
 
 0.09 
 
 
 0.12 
 
 
 0.15 
 
 
 time[ms] 
 
 
 Velocity[m/s] 
 
<polyline clip-path="url(#clip3102)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,367.115 39.9709,367.116 40.5717,367.12 41.1725,367.126 41.7734,367.135 42.3742,367.147 42.975,367.164 43.5758,367.184 44.1767,367.208 44.7775,367.236 
 45.3783,367.268 45.9791,367.302 46.5799,367.34 47.1808,367.38 47.7816,367.423 48.3824,367.468 48.9832,367.515 49.584,367.563 50.1849,367.613 50.7857,367.663 
 51.3865,367.714 51.9873,367.765 52.5882,367.815 53.189,367.866 53.7898,367.915 54.3906,367.964 54.9914,368.012 55.5923,368.058 56.1931,368.103 56.7939,368.146 
 57.3947,368.187 57.9955,368.226 58.5964,368.263 59.1972,368.297 59.798,368.329 60.3988,368.359 60.9997,368.386 61.6005,368.41 62.2013,368.432 62.8021,368.451 
 63.4029,368.467 64.0038,368.48 64.6046,368.49 65.2054,368.498 65.8062,368.502 66.4071,368.504 67.0079,368.503 67.6087,368.499 68.2095,368.492 68.8103,368.482 
 69.4112,368.469 70.012,368.454 70.6128,368.436 71.2136,368.415 71.8144,368.391 72.4153,368.365 73.0161,368.336 73.6169,368.304 74.2177,368.27 74.8186,368.233 
 75.4194,368.194 76.0202,368.152 76.621,368.108 77.2218,368.061 77.8227,368.012 78.4235,367.96 79.0243,367.907 79.6251,367.85 80.226,367.792 80.8268,367.731 
 81.4276,367.668 82.0284,367.603 82.6292,367.536 83.2301,367.466 83.8309,367.394 84.4317,367.321 85.0325,367.245 85.6333,367.167 86.2342,367.087 86.835,367.005 
 87.4358,366.921 88.0366,366.834 88.6375,366.746 89.2383,366.656 89.8391,366.564 90.4399,366.47 91.0407,366.374 91.6416,366.276 92.2424,366.176 92.8432,366.074 
 93.444,365.97 94.0448,365.864 94.6457,365.756 95.2465,365.646 95.8473,365.535 96.4481,365.421 97.049,365.305 97.6498,365.188 98.2506,365.068 98.8514,364.947 
 99.4522,364.823 100.053,364.698 100.654,364.57 101.255,364.441 101.856,364.309 102.456,364.176 103.057,364.04 103.658,363.902 104.259,363.762 104.86,363.621 
 105.46,363.477 106.061,363.33 106.662,363.182 107.263,363.032 107.864,362.879 108.465,362.724 109.065,362.567 109.666,362.408 110.267,362.246 110.868,362.082 
 111.469,361.916 112.069,361.747 112.67,361.576 113.271,361.403 113.872,361.227 114.473,361.048 115.074,360.867 115.674,360.684 116.275,360.498 116.876,360.309 
 117.477,360.118 118.078,359.924 118.679,359.727 119.279,359.527 119.88,359.325 120.481,359.12 121.082,358.911 121.683,358.7 122.283,358.486 122.884,358.269 
 123.485,358.049 124.086,357.826 124.687,357.599 125.288,357.369 125.888,357.136 126.489,356.9 127.09,356.66 127.691,356.417 128.292,356.17 128.893,355.92 
 129.493,355.666 130.094,355.409 130.695,355.148 131.296,354.883 131.897,354.614 132.497,354.341 133.098,354.064 133.699,353.783 134.3,353.499 134.901,353.209 
 135.502,352.916 136.102,352.618 136.703,352.316 137.304,352.01 137.905,351.699 138.506,351.383 139.106,351.063 139.707,350.738 140.308,350.408 140.909,350.073 
 141.51,349.733 142.111,349.389 142.711,349.039 143.312,348.683 143.913,348.323 144.514,347.956 145.115,347.585 145.716,347.208 146.316,346.825 146.917,346.436 
 147.518,346.042 148.119,345.641 148.72,345.235 149.32,344.822 149.921,344.403 150.522,343.978 151.123,343.546 151.724,343.108 152.325,342.663 152.925,342.211 
 153.526,341.752 154.127,341.287 154.728,340.814 155.329,340.334 155.929,339.847 156.53,339.353 157.131,338.851 157.732,338.341 158.333,337.823 158.934,337.298 
 159.534,336.764 160.135,336.223 160.736,335.673 161.337,335.114 161.938,334.547 162.539,333.971 163.139,333.387 163.74,332.792 164.341,332.189 164.942,331.576 
 165.543,330.953 166.143,330.32 166.744,329.676 167.345,329.022 167.946,328.358 168.547,327.682 169.148,326.994 169.748,326.295 170.349,325.584 170.95,324.861 
 171.551,324.125 172.152,323.376 172.752,322.614 173.353,321.839 173.954,321.049 1

In [9]:
plot(t,xa_plot_log,lab="Acceleration x",linewidth=3)
plot!(xlab="time[ms]",ylab="Acceleration[m/s]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 -0.6 
 
 
 -0.3 
 
 
 0.0 
 
 
 0.3 
 
 
 0.6 
 
 
 time[ms] 
 
 
 Acceleration[m/s] 
 
<polyline clip-path="url(#clip3202)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,189.222 39.9709,189.305 40.5717,189.408 41.1725,189.539 41.7734,189.685 42.3742,189.842 42.975,190.003 43.5758,190.166 44.1767,190.326 44.7775,190.48 
 45.3783,190.625 45.9791,190.761 46.5799,190.884 47.1808,190.995 47.7816,191.093 48.3824,191.176 48.9832,191.245 49.584,191.3 50.1849,191.341 50.7857,191.368 
 51.3865,191.382 51.9873,191.383 52.5882,191.372 53.189,191.349 53.7898,191.316 54.3906,191.272 54.9914,191.218 55.5923,191.156 56.1931,191.086 56.7939,191.008 
 57.3947,190.924 57.9955,190.833 58.5964,190.738 59.1972,190.637 59.798,190.532 60.3988,190.424 60.9997,190.312 61.6005,190.197 62.2013,190.081 62.8021,189.962 
 63.4029,189.842 64.0038,189.721 64.6046,189.6 65.2054,189.477 65.8062,189.355 66.4071,189.233 67.0079,189.11 67.6087,188.989 68.2095,188.868 68.8103,188.747 
 69.4112,188.628 70.012,188.509 70.6128,188.392 71.2136,188.276 71.8144,188.161 72.4153,188.047 73.0161,187.934 73.6169,187.823 74.2177,187.714 74.8186,187.605 
 75.4194,187.498 76.0202,187.393 76.621,187.289 77.2218,187.186 77.8227,187.084 78.4235,186.984 79.0243,186.885 79.6251,186.787 80.226,186.691 80.8268,186.595 
 81.4276,186.501 82.0284,186.408 82.6292,186.315 83.2301,186.224 83.8309,186.133 84.4317,186.044 85.0325,185.955 85.6333,185.867 86.2342,185.78 86.835,185.693 
 87.4358,185.607 88.0366,185.521 88.6375,185.436 89.2383,185.352 89.8391,185.268 90.4399,185.184 91.0407,185.1 91.6416,185.017 92.2424,184.934 92.8432,184.851 
 93.444,184.768 94.0448,184.685 94.6457,184.603 95.2465,184.52 95.8473,184.437 96.4481,184.354 97.049,184.271 97.6498,184.187 98.2506,184.103 98.8514,184.019 
 99.4522,183.935 100.053,183.85 100.654,183.765 101.255,183.68 101.856,183.593 102.456,183.507 103.057,183.419 103.658,183.331 104.259,183.243 104.86,183.153 
 105.46,183.063 106.061,182.972 106.662,182.88 107.263,182.788 107.864,182.694 108.465,182.6 109.065,182.504 109.666,182.408 110.267,182.31 110.868,182.211 
 111.469,182.111 112.069,182.01 112.67,181.908 113.271,181.804 113.872,181.699 114.473,181.593 115.074,181.485 115.674,181.376 116.275,181.265 116.876,181.153 
 117.477,181.039 118.078,180.923 118.679,180.806 119.279,180.688 119.88,180.567 120.481,180.445 121.082,180.321 121.683,180.195 122.283,180.067 122.884,179.937 
 123.485,179.805 124.086,179.671 124.687,179.535 125.288,179.397 125.888,179.257 126.489,179.114 127.09,178.97 127.691,178.822 128.292,178.673 128.893,178.521 
 129.493,178.367 130.094,178.21 130.695,178.05 131.296,177.888 131.897,177.723 132.497,177.556 133.098,177.386 133.699,177.213 134.3,177.037 134.901,176.858 
 135.502,176.676 136.102,176.492 136.703,176.304 137.304,176.113 137.905,175.919 138.506,175.722 139.106,175.521 139.707,175.317 140.308,175.11 140.909,174.899 
 141.51,174.685 142.111,174.468 142.711,174.247 143.312,174.022 143.913,173.794 144.514,173.562 145.115,173.326 145.716,173.086 146.316,172.843 146.917,172.596 
 147.518,172.344 148.119,172.089 148.72,171.83 149.32,171.566 149.921,171.299 150.522,171.027 151.123,170.751 151.724,170.471 152.325,170.186 152.925,169.897 
 153.526,169.603 154.127,169.305 154.728,169.002 155.329,168.694 155.929,168.381 156.53,168.063 157.131,167.741 157.732,167.413 158.333,167.079 158.934,166.741 
 159.534,166.396 160.135,166.046 160.736,165.69 161.337,165.328 161.938,164.958 162.539,164.58 163.139,164.194 163.74,163.799 164.341,163.394 164.942,162.981 
 165.543,162.557 166.143,162.123 166.744,161.679 167.345,161.224 167.946,160.758 168.547,160.28 169.148,159.79 169.748,159.288 170.349,158.774 170.95,158.246 
 171.551,157.706 172.152,157.151 172.752,156.583 173.353,155.999 173.954,155.401 174.555,154.786 1

In [10]:
plot(t,yv_plot_log,lab="Velocity y",linewidth=3)
plot!(xlab="time[ms]",ylab="Velocity[m/s]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 -0.20 
 
 
 -0.15 
 
 
 -0.10 
 
 
 -0.05 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 time[ms] 
 
 
 Velocity[m/s] 
 
<polyline clip-path="url(#clip3302)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,127.143 39.9709,127.143 40.5717,127.142 41.1725,127.14 41.7734,127.136 42.3742,127.129 42.975,127.12 43.5758,127.108 44.1767,127.092 44.7775,127.073 
 45.3783,127.05 45.9791,127.024 46.5799,126.995 47.1808,126.962 47.7816,126.926 48.3824,126.887 48.9832,126.845 49.584,126.8 50.1849,126.753 50.7857,126.703 
 51.3865,126.651 51.9873,126.598 52.5882,126.542 53.189,126.484 53.7898,126.425 54.3906,126.365 54.9914,126.303 55.5923,126.241 56.1931,126.177 56.7939,126.112 
 57.3947,126.047 57.9955,125.982 58.5964,125.915 59.1972,125.849 59.798,125.782 60.3988,125.715 60.9997,125.648 61.6005,125.58 62.2013,125.513 62.8021,125.446 
 63.4029,125.378 64.0038,125.311 64.6046,125.244 65.2054,125.177 65.8062,125.11 66.4071,125.043 67.0079,124.977 67.6087,124.911 68.2095,124.844 68.8103,124.778 
 69.4112,124.713 70.012,124.647 70.6128,124.581 71.2136,124.516 71.8144,124.45 72.4153,124.385 73.0161,124.32 73.6169,124.255 74.2177,124.19 74.8186,124.125 
 75.4194,124.06 76.0202,123.994 76.621,123.929 77.2218,123.864 77.8227,123.798 78.4235,123.733 79.0243,123.667 79.6251,123.601 80.226,123.535 80.8268,123.468 
 81.4276,123.402 82.0284,123.334 82.6292,123.267 83.2301,123.199 83.8309,123.131 84.4317,123.062 85.0325,122.993 85.6333,122.923 86.2342,122.853 86.835,122.782 
 87.4358,122.711 88.0366,122.638 88.6375,122.566 89.2383,122.492 89.8391,122.418 90.4399,122.343 91.0407,122.268 91.6416,122.191 92.2424,122.114 92.8432,122.036 
 93.444,121.957 94.0448,121.877 94.6457,121.796 95.2465,121.714 95.8473,121.631 96.4481,121.547 97.049,121.462 97.6498,121.376 98.2506,121.288 98.8514,121.2 
 99.4522,121.11 100.053,121.019 100.654,120.927 101.255,120.834 101.856,120.739 102.456,120.643 103.057,120.545 103.658,120.446 104.259,120.346 104.86,120.244 
 105.46,120.14 106.061,120.035 106.662,119.929 107.263,119.82 107.864,119.71 108.465,119.599 109.065,119.485 109.666,119.37 110.267,119.253 110.868,119.135 
 111.469,119.014 112.069,118.892 112.67,118.767 113.271,118.641 113.872,118.512 114.473,118.382 115.074,118.249 115.674,118.114 116.275,117.977 116.876,117.838 
 117.477,117.696 118.078,117.553 118.679,117.407 119.279,117.258 119.88,117.107 120.481,116.953 121.082,116.797 121.683,116.639 122.283,116.478 122.884,116.314 
 123.485,116.147 124.086,115.978 124.687,115.806 125.288,115.631 125.888,115.453 126.489,115.272 127.09,115.088 127.691,114.901 128.292,114.711 128.893,114.517 
 129.493,114.321 130.094,114.121 130.695,113.918 131.296,113.711 131.897,113.501 132.497,113.287 133.098,113.07 133.699,112.849 134.3,112.625 134.901,112.397 
 135.502,112.164 136.102,111.928 136.703,111.688 137.304,111.444 137.905,111.196 138.506,110.944 139.106,110.687 139.707,110.427 140.308,110.161 140.909,109.892 
 141.51,109.617 142.111,109.339 142.711,109.055 143.312,108.767 143.913,108.474 144.514,108.176 145.115,107.873 145.716,107.565 146.316,107.252 146.917,106.934 
 147.518,106.61 148.119,106.281 148.72,105.947 149.32,105.607 149.921,105.261 150.522,104.91 151.123,104.553 151.724,104.19 152.325,103.821 152.925,103.446 
 153.526,103.065 154.127,102.677 154.728,102.283 155.329,101.883 155.929,101.476 156.53,101.062 157.131,100.642 157.732,100.214 158.333,99.7802 158.934,99.3389 
 159.534,98.8903 160.135,98.4344 160.736,97.9711 161.337,97.5003 161.938,97.0217 162.539,96.5354 163.139,96.0411 163.74,95.5389 164.341,95.0285 164.942,94.5099 
 165.543,93.9829 166.143,93.4474 166.744,92.9033 167.345,92.3503 167.946,91.7885 168.547,91.2175 169.148,90.6373 169.748,90.0476 170.349,89.4482 170.95,88.839 
 171.551,88.2197 172.152,87.5901 172.752,86.95 173.353,86.2991

In [11]:
plot(t,ya_plot_log,lab="Acceleration y",linewidth=3)
plot!(xlab="time[ms]",ylab="Acceleration[m/s]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 time[ms] 
 
 
 Acceleration[m/s] 
 
<polyline clip-path="url(#clip3402)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,186.011 39.9709,186.011 40.5717,185.94 41.1725,185.832 41.7734,185.688 42.3742,185.519 42.975,185.331 43.5758,185.128 44.1767,184.917 44.7775,184.701 
 45.3783,184.484 45.9791,184.269 46.5799,184.058 47.1808,183.854 47.7816,183.657 48.3824,183.469 48.9832,183.291 49.584,183.124 50.1849,182.967 50.7857,182.822 
 51.3865,182.688 51.9873,182.565 52.5882,182.453 53.189,182.352 53.7898,182.261 54.3906,182.18 54.9914,182.109 55.5923,182.046 56.1931,181.991 56.7939,181.944 
 57.3947,181.905 57.9955,181.872 58.5964,181.845 59.1972,181.824 59.798,181.808 60.3988,181.796 60.9997,181.788 61.6005,181.784 62.2013,181.783 62.8021,181.785 
 63.4029,181.789 64.0038,181.795 64.6046,181.802 65.2054,181.811 65.8062,181.82 66.4071,181.831 67.0079,181.841 67.6087,181.852 68.2095,181.862 68.8103,181.873 
 69.4112,181.883 70.012,181.892 70.6128,181.9 71.2136,181.908 71.8144,181.914 72.4153,181.919 73.0161,181.923 73.6169,181.926 74.2177,181.927 74.8186,181.926 
 75.4194,181.924 76.0202,181.921 76.621,181.915 77.2218,181.908 77.8227,181.9 78.4235,181.889 79.0243,181.877 79.6251,181.863 80.226,181.847 80.8268,181.829 
 81.4276,181.809 82.0284,181.788 82.6292,181.764 83.2301,181.739 83.8309,181.712 84.4317,181.683 85.0325,181.652 85.6333,181.619 86.2342,181.584 86.835,181.548 
 87.4358,181.509 88.0366,181.469 88.6375,181.427 89.2383,181.383 89.8391,181.337 90.4399,181.289 91.0407,181.239 91.6416,181.188 92.2424,181.134 92.8432,181.079 
 93.444,181.021 94.0448,180.962 94.6457,180.901 95.2465,180.838 95.8473,180.774 96.4481,180.707 97.049,180.638 97.6498,180.567 98.2506,180.495 98.8514,180.42 
 99.4522,180.344 100.053,180.266 100.654,180.185 101.255,180.103 101.856,180.019 102.456,179.932 103.057,179.844 103.658,179.753 104.259,179.661 104.86,179.566 
 105.46,179.469 106.061,179.371 106.662,179.27 107.263,179.167 107.864,179.061 108.465,178.954 109.065,178.844 109.666,178.732 110.267,178.618 110.868,178.501 
 111.469,178.382 112.069,178.261 112.67,178.138 113.271,178.012 113.872,177.883 114.473,177.752 115.074,177.619 115.674,177.483 116.275,177.344 116.876,177.203 
 117.477,177.06 118.078,176.913 118.679,176.764 119.279,176.612 119.88,176.458 120.481,176.3 121.082,176.14 121.683,175.977 122.283,175.811 122.884,175.642 
 123.485,175.47 124.086,175.295 124.687,175.116 125.288,174.935 125.888,174.75 126.489,174.562 127.09,174.371 127.691,174.177 128.292,173.979 128.893,173.778 
 129.493,173.573 130.094,173.364 130.695,173.152 131.296,172.937 131.897,172.718 132.497,172.495 133.098,172.268 133.699,172.037 134.3,171.802 134.901,171.564 
 135.502,171.321 136.102,171.074 136.703,170.823 137.304,170.568 137.905,170.309 138.506,170.046 139.106,169.778 139.707,169.505 140.308,169.228 140.909,168.947 
 141.51,168.661 142.111,168.37 142.711,168.075 143.312,167.775 143.913,167.47 144.514,167.16 145.115,166.845 145.716,166.526 146.316,166.201 146.917,165.871 
 147.518,165.536 148.119,165.196 148.72,164.85 149.32,164.499 149.921,164.143 150.522,163.781 151.123,163.414 151.724,163.041 152.325,162.662 152.925,162.277 
 153.526,161.887 154.127,161.49 154.728,161.088 155.329,160.679 155.929,160.264 156.53,159.842 157.131,159.414 157.732,158.979 158.333,158.537 158.934,158.089 
 159.534,157.633 160.135,157.169 160.736,156.698 161.337,156.22 161.938,155.735 162.539,155.243 163.139,154.744 163.74,154.237 164.341,153.723 164.942,153.202 
 165.543,152.672 166.143,152.134 166.744,151.588 167.345,151.032 167.946,150.466 168.547,149.889 169.148,149.302 169.748,148.702 170.349,148.09 170.95,147.465 
 171.551,146.826 172.152,146.172 172.752,145.502 173.353,1

In [22]:
plot(x_plot_log,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
a=[0.25 0.24]
b=[0.15 0.15]
plot!(b,a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.16 
 
 
 0.18 
 
 
 0.20 
 
 
 0.22 
 
 
 0.24 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
<polyline clip-path="url(#clip4502)" style="stroke:#0000ff; stroke-width:3; stroke-opacity:1; fill:none" points="
 73.8307,305.117 73.8307,305.117 73.8307,305.117 73.8307,305.117 73.8305,305.117 73.8304,305.117 73.8301,305.116 73.8296,305.116 73.8291,305.116 73.8283,305.116 
 73.8273,305.116 73.826,305.115 73.8245,305.115 73.8228,305.114 73.8207,305.114 73.8183,305.113 73.8155,305.112 73.8124,305.111 73.809,305.109 73.8052,305.108 
 73.801,305.106 73.7964,305.104 73.7915,305.102 73.7862,305.1 73.7805,305.097 73.7745,305.095 73.7681,305.092 73.7614,305.088 73.7544,305.085 73.747,305.081 
 73.7393,305.077 73.7313,305.073 73.7231,305.069 73.7146,305.064 73.7058,305.06 73.6969,305.055 73.6877,305.049 73.6783,305.044 73.6688,305.038 73.6591,305.032 
 73.6493,305.026 73.6393,305.019 73.6293,305.012 73.6193,305.005 73.6091,304.998 73.599,304.99 73.5889,304.983 73.5788,304.975 73.5687,304.966 73.5587,304.958 
 73.5487,304.949 73.5389,304.94 73.5292,304.931 73.5196,304.922 73.5102,304.912 73.501,304.902 73.4919,304.892 73.4831,304.881 73.4746,304.871 73.4663,304.86 
 73.4583,304.849 73.4505,304.837 73.4431,304.826 73.436,304.814 73.4293,304.802 73.4229,304.79 73.417,304.777 73.4114,304.764 73.4062,304.751 73.4015,304.738 
 73.3973,304.725 73.3935,304.711 73.3901,304.697 73.3873,304.683 73.385,304.668 73.3832,304.654 73.382,304.639 73.3814,304.623 73.3813,304.608 73.3818,304.592 
 73.3829,304.576 73.3846,304.56 73.387,304.543 73.39,304.527 73.3937,304.509 73.3981,304.492 73.4032,304.475 73.4089,304.457 73.4154,304.438 73.4227,304.42 
 73.4307,304.401 73.4394,304.382 73.449,304.363 73.4593,304.343 73.4704,304.323 73.4824,304.303 73.4952,304.283 73.5088,304.262 73.5233,304.241 73.5387,304.219 
 73.555,304.197 73.5722,304.175 73.5904,304.153 73.6094,304.13 73.6294,304.107 73.6504,304.083 73.6724,304.06 73.6954,304.035 73.7193,304.011 73.7444,303.986 
 73.7704,303.961 73.7975,303.935 73.8257,303.909 73.855,303.883 73.8854,303.856 73.9169,303.828 73.9495,303.801 73.9833,303.773 74.0183,303.744 74.0545,303.715 
 74.0918,303.686 74.1304,303.656 74.1703,303.626 74.2114,303.595 74.2537,303.564 74.2974,303.532 74.3424,303.5 74.3887,303.468 74.4363,303.434 74.4854,303.401 
 74.5358,303.367 74.5876,303.332 74.6409,303.297 74.6955,303.261 74.7517,303.225 74.8094,303.188 74.8685,303.151 74.9292,303.113 74.9915,303.074 75.0553,303.035 
 75.1207,302.995 75.1878,302.955 75.2564,302.914 75.3268,302.873 75.3988,302.83 75.4726,302.788 75.5481,302.744 75.6253,302.7 75.7044,302.655 75.7852,302.609 
 75.8679,302.563 75.9525,302.516 76.039,302.468 76.1274,302.42 76.2177,302.37 76.31,302.32 76.4043,302.27 76.5007,302.218 76.5991,302.165 76.6996,302.112 
 76.8023,302.058 76.9071,302.003 77.0141,301.947 77.1233,301.891 77.2348,301.833 77.3485,301.775 77.4646,301.715 77.583,301.655 77.7039,301.593 77.8271,301.531 
 77.9529,301.468 78.0811,301.404 78.2118,301.338 78.3452,301.272 78.4811,301.205 78.6197,301.136 78.761,301.066 78.9051,300.996 79.0519,300.924 79.2015,300.851 
 79.354,300.777 79.5094,300.702 79.6677,300.625 79.8291,300.547 79.9934,300.468 80.1609,300.388 80.3315,300.306 80.5052,300.224 80.6822,300.139 80.8625,300.054 
 81.0461,299.967 81.233,299.878 81.4234,299.789 81.6173,299.697 81.8147,299.605 82.0157,299.511 82.2203,299.415 82.4286,299.318 82.6407,299.219 82.8566,299.119 
 83.0763,299.017 83.3,298.913 83.5276,298.808 83.7593,298.701 83.9952,298.592 84.2352,298.481 84.4794,298.369 84.7279,298.255 84.9808,298.139 85.2382,298.021 
 85.5001,297.901 85.7665,297.78 86.0376,297.656 86.3135,297.53 86.5942,297.403 86.8797,297.273 87.1703,297.141 87.4659,297.007 87.7667,296.871 88.0727,296.733 
 88.384,296.592 88.7007,296.45 89.023,296.304 89.3509,296.157 89.6845,296.007 90.0239,295.855 90.3693,295.7 90.7206,295.543 91.0782,295.383 9